# 📊 Credit Risk Analysis

This project aims to predict credit risk by determining which customers are likely to pay their loans on time and which are not. We'll employ various data science techniques and methodologies.

## 🎯 Objectives

We will cover the following concepts:

1. **🔍 Exploratory Data Analysis (EDA)**:
   - Analyze the dataset to uncover patterns, spot anomalies, and check assumptions using statistical summaries and visualizations.

2. **🛠️ Data Preprocessing**:
   - Clean the data, handle missing values, encode categorical variables, normalize data, and split into training and testing sets.

3. **⭐ Feature Importance**:
   - Identify key features influencing `loan_status` to improve model performance and interpretability.

4. **🔽 Dimensionality Reduction**:
   - Use techniques like PCA to reduce the number of features while retaining essential information, speeding up model training, and reducing overfitting.

5. **🤖 Predictive Modeling**:
   - Build various models (logistic regression, decision trees, random forests, gradient boosting) to predict the target variable `loan_status`.

6. **⚙️ Hyperparameter Optimization**:
   - Perform hyperparameter tuning with Optuna to find the best parameters for our models.

7. **🧪 Model Testing**:
   - Evaluate models using metrics like accuracy, precision, recall, F1 score, and AUC-ROC to test performance on unseen data.

## 🎯 Goal

Predict the `loan_status` to determine the likelihood of customers paying their loans on time. This helps financial institutions make informed loan approval decisions and manage risk effectively.


# 📖 Data Dictionary

| Column Name            | Description                                                                                           |
|------------------------|-------------------------------------------------------------------------------------------------------|
| out_prncp_inv          | Remaining outstanding principal for portion of total amount funded by investors                       |
| policy_code            | Publicly available policy_code=1. <br> New products not publicly available policy_code=2                   |
| pub_rec                | Number of derogatory public records                                                                   |
| purpose                | A category provided by the borrower for the loan request.                                             |
| pymnt_plan             | Indicates if a payment plan has been put in place for the loan                                        |
| recoveries             | Post charge off gross recovery                                                                        |
| revol_bal              | Total credit revolving balance                                                                        |
| revol_util             | Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit. |
| sub_grade              | LC assigned loan subgrade                                                                             |
| term                   | The number of payments on the loan. Values are in months and can be either 36 or 60.                  |
| title                  | The loan title provided by the borrower                                                               |
| total_acc              | The total number of credit lines currently in the borrower's credit file                              |
| total_pymnt            | Payments received to date for total amount funded                                                     |
| total_pymnt_inv        | Payments received to date for portion of total amount funded by investors                             |
| total_rec_int          | Interest received to date                                                                             |
| total_rec_late_fee     | Late fees received to date                                                                            |
| total_rec_prncp        | Principal received to date                                                                            |
| url                    | URL for the LC page with listing data.                                                                |
| verified_status_joint  | Indicates if the co-borrowers' joint income was verified by LC, not verified, or if the income source was verified |
| zip_code               | The first 3 numbers of the zip code provided by the borrower in the loan application.                 |
| open_acc_6m            | Number of open trades in last 6 months                                                                |
| open_il_6m             | Number of currently active installment trades                                                         |
| open_il_12m            | Number of installment accounts opened in past 12 months                                               |
| open_il_24m            | Number of installment accounts opened in past 24 months                                               |
| mths_since_rcnt_il     | Months since most recent installment accounts opened                                                  |
| total_bal_il           | Total current balance of all installment accounts                                                     |
| il_util                | Ratio of total current balance to high credit/credit limit on all install acct                        |
| open_rv_12m            | Number of revolving trades opened in past 12 months                                                   |
| open_rv_24m            | Number of revolving trades opened in past 24 months                                                   |
| max_bal_bc             | Maximum current balance owed on all revolving accounts                                                |
| all_util               | Balance to credit limit on all trades                                                                 |
| total_rev_hi_lim       | Total revolving high credit/credit limit                                                              |
| inq_fi                 | Number of personal finance inquiries                                                                  |
| total_cu_tl            | Number of finance trades                                                                              |
| inq_last_12m           | Number of credit inquiries in past 12 months                                                          |
| acc_now_delinq         | The number of accounts on which the borrower is now delinquent.                                       |
| tot_coll_amt           | Total collection amounts ever owed                                                                    |
| tot_cur_bal            | Total current balance of all accounts                                                                 |

#### * Employer Title replaces Employer Name for all loans listed after 9/23/2013



## Some important features

| Column      | Description                                  |
|-------------|----------------------------------------------|
| loan_amnt   | Amount of money requested by the borrower.   |
| int_rate    | Interest rate of the loan.                   |
| grade       | Loan grade with categories A, B, C, D, E, F, G. |
| annual_inc  | Borrower's annual income.                    |
| purpose     | The primary purpose of borrowing.            |
| installment | Monthly amount payments for opted loan.      |
| term        | Duration of the loan until it’s paid off.    |


# 🚀 Importing libraries and getting started

In [1]:
import pandas as pd
import numpy as np

import plotly.express as px
from sklearn import metrics
from sklearn.impute import KNNImputer, SimpleImputer, MissingIndicator

from sklearn.preprocessing import LabelEncoder

import re

In [2]:
data = pd.read_csv('C:/Users/saran/OneDrive/Documents/GitHub/credit-risk/data/loan.csv', low_memory=False)

In [3]:
print(data.shape)
print(data.shape[0] * data.shape[1])

(887379, 74)
65666046


### We have about 887K rows and 74 columns, which amounts to about 65 million data points.

In [4]:
data['loan_status'].unique() # Understanding the target variable

array(['Fully Paid', 'Charged Off', 'Current', 'Default',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Issued'],
      dtype=object)

### Here is what the terms in the target variable mean
| Term                                               | Meaning                                                                                                                                                 |
|----------------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Fully Paid**                                     | The borrower has completely repaid the loan.                                                                                                             |
| **Charged Off**                                    | The lender has given up on collecting the loan because the borrower hasn’t paid for a long time. The loan is considered a loss.                          |
| **Current**                                        | The borrower is making payments on time.                                                                                                                 |
| **Default**                                        | The borrower has stopped making payments for a long period, and the loan is in serious trouble.                                                          |
| **Late (31-120 days)**                             | The borrower has missed payments and is behind by 31 to 120 days.                                                                                        |
| **In Grace Period**                                | The borrower missed a payment, but the late fee hasn’t been applied yet because it’s within an allowed period after the due date.                        |
| **Late (16-30 days)**                              | The borrower is behind on payments by 16 to 30 days.                                                                                                     |
| **Does not meet the credit policy. Status: Fully Paid** | The loan didn’t meet the lender's usual criteria but was still given and has been completely repaid.                                                      |
| **Does not meet the credit policy. Status: Charged Off** | The loan didn’t meet the lender's usual criteria, was still given, but ended up in loss as the borrower didn’t repay.                                      |
| **Issued**                                         | The loan has been approved and the money has been given to the borrower.                                                                                 |


In [5]:
for i in data.columns[2:]:
    print(i)
    print(data[i].head(), '\n')

loan_amnt
0     5000.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: loan_amnt, dtype: float64 

funded_amnt
0     5000.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: funded_amnt, dtype: float64 

funded_amnt_inv
0     4975.0
1     2500.0
2     2400.0
3    10000.0
4     3000.0
Name: funded_amnt_inv, dtype: float64 

term
0     36 months
1     60 months
2     36 months
3     36 months
4     60 months
Name: term, dtype: object 

int_rate
0    10.65
1    15.27
2    15.96
3    13.49
4    12.69
Name: int_rate, dtype: float64 

installment
0    162.87
1     59.83
2     84.33
3    339.31
4     67.79
Name: installment, dtype: float64 

grade
0    B
1    C
2    C
3    C
4    B
Name: grade, dtype: object 

sub_grade
0    B2
1    C4
2    C5
3    C1
4    B5
Name: sub_grade, dtype: object 

emp_title
0                         NaN
1                       Ryder
2                         NaN
3         AIR RESOURCES BOARD
4    University Medical Group
Name: emp_title, dtype: ob

In [6]:
y = data['loan_status']

In [7]:
y.isna().sum()

0

No null values in target column.

In [8]:
X = data.drop(columns='loan_status')

In [9]:
X.shape

(887379, 73)

In [10]:
print('Number of null data points:', X.isna().sum().sum())
print('Percentage of null data:', round( (X.isna().sum().sum() / (X.shape[0]*X.shape[1]) ) * 100, 2), '%')    

Number of null data points: 17998493
Percentage of null data: 27.78 %


In [11]:
for i in X.columns:
    null = ( X[i].isna().sum() / len(X[i]) ) * 100
    print(f'Percentage null values in {i}: {round(null, 2)}%')

Percentage null values in id: 0.0%
Percentage null values in member_id: 0.0%
Percentage null values in loan_amnt: 0.0%
Percentage null values in funded_amnt: 0.0%
Percentage null values in funded_amnt_inv: 0.0%
Percentage null values in term: 0.0%
Percentage null values in int_rate: 0.0%
Percentage null values in installment: 0.0%
Percentage null values in grade: 0.0%
Percentage null values in sub_grade: 0.0%
Percentage null values in emp_title: 5.8%
Percentage null values in emp_length: 5.05%
Percentage null values in home_ownership: 0.0%
Percentage null values in annual_inc: 0.0%
Percentage null values in verification_status: 0.0%
Percentage null values in issue_d: 0.0%
Percentage null values in pymnt_plan: 0.0%
Percentage null values in url: 0.0%
Percentage null values in desc: 85.8%
Percentage null values in purpose: 0.0%
Percentage null values in title: 0.02%
Percentage null values in zip_code: 0.0%
Percentage null values in addr_state: 0.0%
Percentage null values in dti: 0.0%
Per

Many features have a high percentage of null values.<br /> We will drop features with more than 97% null values.<br /> For the remaining features with a high percentage of nulls, we will handle them differently.

In [12]:
dropped_list = [] # list to track dropped features
for i in X.columns:
    null = ( X[i].isna().sum() / len(X[i]) ) * 100 
    if null < 97:
        print(f'Percentage null values in {i}: {round(null, 2)}%')

    if null > 97:
        print(f'Percentage null values in {i}: {round(null, 2)}% -- FEATURE DROPPED')
        dropped_list.append(i)
        X.drop(columns=i, inplace = True)

print('Total features dropped: ', len(dropped_list))

Percentage null values in id: 0.0%
Percentage null values in member_id: 0.0%
Percentage null values in loan_amnt: 0.0%
Percentage null values in funded_amnt: 0.0%
Percentage null values in funded_amnt_inv: 0.0%
Percentage null values in term: 0.0%
Percentage null values in int_rate: 0.0%
Percentage null values in installment: 0.0%
Percentage null values in grade: 0.0%
Percentage null values in sub_grade: 0.0%
Percentage null values in emp_title: 5.8%
Percentage null values in emp_length: 5.05%
Percentage null values in home_ownership: 0.0%
Percentage null values in annual_inc: 0.0%
Percentage null values in verification_status: 0.0%
Percentage null values in issue_d: 0.0%
Percentage null values in pymnt_plan: 0.0%
Percentage null values in url: 0.0%
Percentage null values in desc: 85.8%
Percentage null values in purpose: 0.0%
Percentage null values in title: 0.02%
Percentage null values in zip_code: 0.0%
Percentage null values in addr_state: 0.0%
Percentage null values in dti: 0.0%
Per

In [13]:
X.shape

(887379, 56)

17 feautures have been dropped.

In [14]:
for i in X.columns:
    null = ( X[i].isna().sum() / len(X[i]) ) * 100
    if null > 50:
        print(f'Percentage null values in {i}: {round(null, 2)}%')
    else:
        pass

Percentage null values in desc: 85.8%
Percentage null values in mths_since_last_delinq: 51.2%
Percentage null values in mths_since_last_record: 84.56%
Percentage null values in mths_since_last_major_derog: 75.02%


We will drop the description feature, and impute other features with 0.

In [15]:
X.drop(columns = 'desc', inplace = True)
dropped_list.append('desc')

In [16]:
num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(exclude=['int64', 'float64']).columns

In [17]:
print('NUMERICAL COLUMNS\n')
count_num = 0
count_cat = 0
for i in num_cols:
    null = ( X[i].isna().sum() / len(X[i]) ) * 100
    if null > 0:
        count_num = count_num + 1
        print(f'Percentage null values in {i}: {round(null, 2)}%')
        print(f'Total null values in {i}: {X[i].isna().sum()}\n')
    

print('\nCATEGORICAL COLUMNS\n')
for i in cat_cols:
    null = ( X[i].isna().sum() / len(X[i]) ) * 100
    if null > 0:
        count_cat = count_cat + 1
        print(f'Percentage null values in {i}: {round(null, 2)}%')
        print(f'Total null values in {i}: {X[i].isna().sum()}\n')
    
print(f'Total numeric null features: {count_num}')
print(f'Total categorical null features: {count_cat}')

NUMERICAL COLUMNS

Percentage null values in annual_inc: 0.0%
Total null values in annual_inc: 4

Percentage null values in delinq_2yrs: 0.0%
Total null values in delinq_2yrs: 29

Percentage null values in inq_last_6mths: 0.0%
Total null values in inq_last_6mths: 29

Percentage null values in mths_since_last_delinq: 51.2%
Total null values in mths_since_last_delinq: 454312

Percentage null values in mths_since_last_record: 84.56%
Total null values in mths_since_last_record: 750326

Percentage null values in open_acc: 0.0%
Total null values in open_acc: 29

Percentage null values in pub_rec: 0.0%
Total null values in pub_rec: 29

Percentage null values in revol_util: 0.06%
Total null values in revol_util: 502

Percentage null values in total_acc: 0.0%
Total null values in total_acc: 29

Percentage null values in collections_12_mths_ex_med: 0.02%
Total null values in collections_12_mths_ex_med: 145

Percentage null values in mths_since_last_major_derog: 75.02%
Total null values in mths_s

In [18]:
X.drop(columns = ['id', 'member_id'], inplace = True)
dropped_list.append('id')
dropped_list.append('member_id')
num_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [19]:
num_cols

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate',
       'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp',
       'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp',
       'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal',
       'total_rev_hi_lim'],
      dtype='object')

In [20]:
knn_imp = KNNImputer(add_indicator = True)
simp_imp = SimpleImputer(strategy = 'constant',
                         fill_value = 0.0)
mi = MissingIndicator()

In [21]:
new_data = X.copy()

In [22]:
for i in num_cols: # Adding a missing indicator for each numerical column with nan values
    if new_data[i].isna().sum() > 0:
        temp = mi.fit_transform(new_data[[i]])
        loc = new_data.columns.get_loc(i)
        new_data.insert(loc+1, value = temp, column = f'{i}_MISSING_INDICATOR')

In [23]:
new_data.shape

(887379, 68)

We have added 15 missing indiactors for 15 columns with missing values.

In [24]:
new_data[num_cols].isna().sum()

loan_amnt                           0
funded_amnt                         0
funded_amnt_inv                     0
int_rate                            0
installment                         0
annual_inc                          4
dti                                 0
delinq_2yrs                        29
inq_last_6mths                     29
mths_since_last_delinq         454312
mths_since_last_record         750326
open_acc                           29
pub_rec                            29
revol_bal                           0
revol_util                        502
total_acc                          29
out_prncp                           0
out_prncp_inv                       0
total_pymnt                         0
total_pymnt_inv                     0
total_rec_prncp                     0
total_rec_int                       0
total_rec_late_fee                  0
recoveries                          0
collection_recovery_fee             0
last_pymnt_amnt                     0
collections_

Now we impute missing numerical values.

In [25]:
new_data['annual_inc'] = knn_imp.fit_transform(new_data[['annual_inc']])

In [26]:
new_data['annual_inc'].isna().sum()

0

In [27]:
for i in num_cols:
    if new_data[i].isna().sum() > 0:
        new_data[i] = simp_imp.fit_transform(new_data[[i]])
        

In [28]:
new_data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,policy_code,application_type,acc_now_delinq,acc_now_delinq_MISSING_INDICATOR,tot_coll_amt,tot_coll_amt_MISSING_INDICATOR,tot_cur_bal,tot_cur_bal_MISSING_INDICATOR,total_rev_hi_lim,total_rev_hi_lim_MISSING_INDICATOR
0,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
1,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
2,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
3,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
4,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True


In [29]:
new_data[num_cols].isna().sum()

loan_amnt                      0
funded_amnt                    0
funded_amnt_inv                0
int_rate                       0
installment                    0
annual_inc                     0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
mths_since_last_delinq         0
mths_since_last_record         0
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                     0
total_acc                      0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
total_rec_late_fee             0
recoveries                     0
collection_recovery_fee        0
last_pymnt_amnt                0
collections_12_mths_ex_med     0
mths_since_last_major_derog    0
policy_code                    0
acc_now_delinq                 0
tot_coll_a

We have imputed missing values for all numeric columns and added corresponding missing indicator columns.

In [30]:
new_data[cat_cols].head()

,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,url,purpose,title,zip_code,addr_state,earliest_cr_line,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type
0,36 months,B,B2,NaN,10+ years,RENT,Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,credit_card,Computer,860xx,AZ,Jan-1985,f,Jan-2015,NaN,Jan-2016,INDIVIDUAL
1,60 months,C,C4,Ryder,< 1 year,RENT,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,car,bike,309xx,GA,Apr-1999,f,Apr-2013,NaN,Sep-2013,INDIVIDUAL
2,36 months,C,C5,NaN,10+ years,RENT,Not Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,small_business,real estate business,606xx,IL,Nov-2001,f,Jun-2014,NaN,Jan-2016,INDIVIDUAL
3,36 months,C,C1,AIR RESOURCES BOARD,10+ years,RENT,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,other,personel,917xx,CA,Feb-1996,f,Jan-2015,NaN,Jan-2015,INDIVIDUAL
4,60 months,B,B5,University Medical Group,1 year,RENT,Source Verified,Dec-2011,n,https://www.lendingclub.com/browse/loanDetail....,other,Personal,972xx,OR,Jan-1996,f,Jan-2016,Feb-2016,Jan-2016,INDIVIDUAL


In [31]:
new_data[cat_cols].dtypes

term                   object
grade                  object
sub_grade              object
emp_title              object
emp_length             object
home_ownership         object
verification_status    object
issue_d                object
pymnt_plan             object
url                    object
purpose                object
title                  object
zip_code               object
addr_state             object
earliest_cr_line       object
initial_list_status    object
last_pymnt_d           object
next_pymnt_d           object
last_credit_pull_d     object
application_type       object
dtype: object

In [32]:
try:
    for i in cat_cols:
        print(np.unique(new_data[i]))
except TypeError: 
    print(TypeError)

[' 36 months' ' 60 months']
['A' 'B' 'C' 'D' 'E' 'F' 'G']
['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4'
 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3'
 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']
<class 'TypeError'>


_Object_ datatype does not gurantee that the data included is of _str_ type. Object generally refers to mixed data type. We can verify this below:

In [33]:
for i in cat_cols:
    print(type(new_data[i].loc[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'float'>
<class 'str'>
<class 'str'>


Above code gave us error for the 3rd feature in the cat_cols list. The 3rd feature's first value is of type _float_. We can typecast all categorical values as _str_.

In [34]:
new_data[cat_cols] = new_data[cat_cols].astype(str)

In [35]:
new_data[cat_cols].dtypes

term                   object
grade                  object
sub_grade              object
emp_title              object
emp_length             object
home_ownership         object
verification_status    object
issue_d                object
pymnt_plan             object
url                    object
purpose                object
title                  object
zip_code               object
addr_state             object
earliest_cr_line       object
initial_list_status    object
last_pymnt_d           object
next_pymnt_d           object
last_credit_pull_d     object
application_type       object
dtype: object

Since _Object_ datatype does not gurantee string, we have to use the type() function to verify.

In [36]:
for i in cat_cols:
    print(type(new_data[i].loc[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


All the categorical features have been converted into _str_ now. We can proceed further.

In [37]:
for i in cat_cols:
    print(f'Feature: {i}\nUnique values: {new_data[i].nunique()}\n{np.unique(new_data[i])}\n')

Feature: term
Unique values: 2
[' 36 months' ' 60 months']

Feature: grade
Unique values: 7
['A' 'B' 'C' 'D' 'E' 'F' 'G']

Feature: sub_grade
Unique values: 35
['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4'
 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3'
 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']

Feature: emp_title
Unique values: 299272
[' \tAdv Mtr Proj Fld Rep' '       NSA Industries llc'
 '    MOTHER  THERESA  HOME' ... 'zulily' 'Å\x9ar.  Technical Illustrator'
 '\u200bLicense Compliance Investigator']

Feature: emp_length
Unique values: 12
['1 year' '10+ years' '2 years' '3 years' '4 years' '5 years' '6 years'
 '7 years' '8 years' '9 years' '< 1 year' 'nan']

Feature: home_ownership
Unique values: 6
['ANY' 'MORTGAGE' 'NONE' 'OTHER' 'OWN' 'RENT']

Feature: verification_status
Unique values: 3
['Not Verified' 'Source Verified' 'Verified']

Feature: issue_d
Unique values: 103
['Apr-2008' 'Apr-2009' 'Apr-2010' 'Apr-2011' 'Apr-2012' 'Apr-2013'
 

Drop the emp_title, url, and title features because they won't add useful information to the machine learning model.

In [38]:
new_data.drop(columns = ['emp_title', 'url', 'title'], inplace = True)

dropped_list.append('emp_title')
dropped_list.append('url')
dropped_list.append('title')

In [39]:
new_data.shape

(887379, 65)

In [40]:
for i in cat_cols:
    if i not in new_data.columns:
        continue
    else:
        print(f'Feature: {i}\nUnique values: {new_data[i].nunique()}\nNaN values: {new_data[i].isna().sum()}\n{np.unique(new_data[i])}\n')

Feature: term
Unique values: 2
NaN values: 0
[' 36 months' ' 60 months']

Feature: grade
Unique values: 7
NaN values: 0
['A' 'B' 'C' 'D' 'E' 'F' 'G']

Feature: sub_grade
Unique values: 35
NaN values: 0
['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4'
 'C5' 'D1' 'D2' 'D3' 'D4' 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3'
 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']

Feature: emp_length
Unique values: 12
NaN values: 0
['1 year' '10+ years' '2 years' '3 years' '4 years' '5 years' '6 years'
 '7 years' '8 years' '9 years' '< 1 year' 'nan']

Feature: home_ownership
Unique values: 6
NaN values: 0
['ANY' 'MORTGAGE' 'NONE' 'OTHER' 'OWN' 'RENT']

Feature: verification_status
Unique values: 3
NaN values: 0
['Not Verified' 'Source Verified' 'Verified']

Feature: issue_d
Unique values: 103
NaN values: 0
['Apr-2008' 'Apr-2009' 'Apr-2010' 'Apr-2011' 'Apr-2012' 'Apr-2013'
 'Apr-2014' 'Apr-2015' 'Aug-2007' 'Aug-2008' 'Aug-2009' 'Aug-2010'
 'Aug-2011' 'Aug-2012' 'Aug-2013' 'Aug-2014' 'Aug-

In [41]:
new_data[new_data['emp_length'] == 'nan']

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,...,policy_code,application_type,acc_now_delinq,acc_now_delinq_MISSING_INDICATOR,tot_coll_amt,tot_coll_amt_MISSING_INDICATOR,tot_cur_bal,tot_cur_bal_MISSING_INDICATOR,total_rev_hi_lim,total_rev_hi_lim_MISSING_INDICATOR
168,2500.0,2500.0,2500.0,36 months,7.51,77.78,A,A3,nan,OWN,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
323,18000.0,18000.0,18000.0,60 months,17.58,452.98,D,D4,nan,MORTGAGE,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
394,4000.0,4000.0,4000.0,36 months,11.71,132.31,B,B3,nan,RENT,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
422,1000.0,1000.0,1000.0,36 months,14.65,34.50,C,C3,nan,RENT,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
439,8250.0,8250.0,8250.0,36 months,7.51,256.67,A,A3,nan,MORTGAGE,...,1.0,INDIVIDUAL,0.0,False,0.0,True,0.0,True,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
887321,20000.0,20000.0,20000.0,60 months,8.67,411.98,B,B1,nan,MORTGAGE,...,1.0,INDIVIDUAL,0.0,False,0.0,False,111924.0,False,54500.0,False
887337,19600.0,19600.0,19600.0,60 months,14.31,459.22,C,C4,nan,OWN,...,1.0,INDIVIDUAL,0.0,False,0.0,False,141120.0,False,233000.0,False
887355,26950.0,26950.0,26950.0,36 months,8.19,846.88,A,A5,nan,RENT,...,1.0,INDIVIDUAL,0.0,False,0.0,False,41650.0,False,129100.0,False
887359,25000.0,25000.0,25000.0,60 months,15.59,602.52,D,D1,nan,OWN,...,1.0,INDIVIDUAL,0.0,False,0.0,False,13860.0,False,24400.0,False


In [137]:
test = new_data[['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']].copy()
test.head(10)

,issue_d,earliest_cr_line,last_pymnt_d,next_pymnt_d,last_credit_pull_d
0,Dec-2011,Jan-1985,Jan-2015,nan,Jan-2016
1,Dec-2011,Apr-1999,Apr-2013,nan,Sep-2013
2,Dec-2011,Nov-2001,Jun-2014,nan,Jan-2016
3,Dec-2011,Feb-1996,Jan-2015,nan,Jan-2015
4,Dec-2011,Jan-1996,Jan-2016,Feb-2016,Jan-2016
5,Dec-2011,Nov-2004,Jan-2015,nan,Sep-2015
6,Dec-2011,Jul-2005,Jan-2016,Feb-2016,Jan-2016
7,Dec-2011,Jan-2007,Jan-2015,nan,Dec-2014
8,Dec-2011,Apr-2004,Apr-2012,nan,Aug-2012
9,Dec-2011,Sep-2004,Nov-2012,nan,Mar-2013


In [138]:
issue_d_month = []
issue_d_year = []

earliest_cr_line_month = []
earliest_cr_line_year = []

last_pymnt_d_month = []
last_pymnt_d_year = []

next_pymnt_d_month = []
next_pymnt_d_year = []

last_credit_pull_d_month = []
last_credit_pull_d_year = []

In [139]:
def month_num(mm):

    month = mm.lower()
    if month == 'jan':
        return 1
    
    elif month == 'feb':
        return 2
        
    elif month == 'mar':
        return 3
        
    elif month == 'apr':
        return 4
        
    elif month == 'may':
        return 5
        
    elif month == 'jun':
        return 6
        
    elif month == 'jul':
        return 7
        
    elif month == 'aug':
        return 8
        
    elif month == 'sept':
        return 9
        
    elif month == 'oct':
        return 10
        
    elif month == 'nov':
        return 11
        
    elif month == 'dec':
        return 12

    else :
        return 0

In [140]:
for i in test.columns:
    for j in range(0, test.shape[0]):
        if test[i][j] != 'nan':
            
            match = re.match(r'([A-Za-z]{3})-(\d{4})', test[i][j])

            month = match.group(1)
            month = month_num(month)

            year = match.group(2)

            if i == 'issue_d':
                issue_d_month.append(month)
                issue_d_year.append(int(year))
            
            elif i == 'earliest_cr_line':
                earliest_cr_line_month.append(month)
                earliest_cr_line_year.append(int(year))

            elif i == 'last_pymnt_d':
                last_pymnt_d_month.append(month)
                last_pymnt_d_year.append(int(year))

            elif i == 'next_pymnt_d':
                next_pymnt_d_month.append(month)
                next_pymnt_d_year.append(int(year))

            elif i == 'last_credit_pull_d':
                last_credit_pull_d_month.append(month)
                last_credit_pull_d_year.append(int(year))

        else:
            month = 0
            year = 0
            
            if i == 'issue_d':
                issue_d_month.append(month)
                issue_d_year.append(int(year))
            
            elif i == 'earliest_cr_line':
                earliest_cr_line_month.append(month)
                earliest_cr_line_year.append(int(year))

            elif i == 'last_pymnt_d':
                last_pymnt_d_month.append(month)
                last_pymnt_d_year.append(int(year))

            elif i == 'next_pymnt_d':
                next_pymnt_d_month.append(month)
                next_pymnt_d_year.append(int(year))

            elif i == 'last_credit_pull_d':
                last_credit_pull_d_month.append(month)
                last_credit_pull_d_year.append(int(year))
            

In [141]:
loc_issue_d = test.columns.get_loc('issue_d')
test.insert(loc = loc_issue_d+1, column='issue_d_month', value = issue_d_month)
test.insert(loc = loc_issue_d+2, column='issue_d_year', value = issue_d_year)

loc_earliest_cr_line = test.columns.get_loc('earliest_cr_line')
test.insert(loc = loc_earliest_cr_line+1, column='earliest_cr_line_month', value = earliest_cr_line_month)
test.insert(loc = loc_earliest_cr_line+2, column='earliest_cr_line_year', value = earliest_cr_line_year)

loc_last_pymnt_d = test.columns.get_loc('last_pymnt_d')
test.insert(loc = loc_last_pymnt_d+1, column='last_pymnt_d_month', value = last_pymnt_d_month)
test.insert(loc = loc_last_pymnt_d+2, column='last_pymnt_d_year', value = last_pymnt_d_year)

loc_next_pymnt_d = test.columns.get_loc('next_pymnt_d')
test.insert(loc = loc_next_pymnt_d+1, column='next_pymnt_d_month', value = next_pymnt_d_month)
test.insert(loc = loc_next_pymnt_d+2, column='next_pymnt_d_year', value = next_pymnt_d_year)

loc_last_credit_pull_d = test.columns.get_loc('last_credit_pull_d')
test.insert(loc = loc_last_credit_pull_d+1, column='last_credit_pull_d_month', value = last_credit_pull_d_month)
test.insert(loc = loc_last_credit_pull_d+2, column='last_credit_pull_d_year', value = last_credit_pull_d_year)

test.drop(columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'], inplace = True)

In [144]:
test.drop(columns = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d'], inplace = True)
test.head()

,issue_d_month,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,next_pymnt_d_month,next_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year
0,12,2011,1,1985,1,2015,0,0,1,2016
1,12,2011,4,1999,4,2013,0,0,0,2013
2,12,2011,11,2001,6,2014,0,0,1,2016
3,12,2011,2,1996,1,2015,0,0,1,2015
4,12,2011,1,1996,1,2016,2,2016,1,2016
